[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-home-credit-default-risk/blob/master/prep.ipynb)

In [0]:
import pandas as pd
import numpy as np
import os
import dotenv
import gc
from tqdm import tqdm 
from sklearn.feature_extraction.text import CountVectorizer

dotenv.load_dotenv('.env')

True

In [0]:
types = ['train', 'test']
dfs = {t:pd.read_csv('./raw/application_%s.csv'%t) for t in types}
bureau = pd.read_csv('./raw/bureau.csv')
bureau_balance = pd.read_csv('./raw/bureau_balance.csv')
prev_app = pd.read_csv('./raw/previous_application.csv')
pos_cash = pd.read_csv('./raw/POS_CASH_balance.csv')
credit_card_balance = pd.read_csv('./raw/credit_card_balance.csv')
previous_application = pd.read_csv('./raw/previous_application.csv')
installments_payments = pd.read_csv('./raw/installments_payments.csv')

In [0]:
for t in types:
  df = dfs[t]
  docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
  live = [_f for _f in df.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
    
  df['NEW_CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
  df['NEW_CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
  
  inc_by_org = df[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']
  
  df['NEW_DOC_IND_AVG'] = df[docs].mean(axis=1)
  df['NEW_DOC_IND_STD'] = df[docs].std(axis=1)
  df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1)
  df['NEW_LIVE_IND_SUM'] = df[live].sum(axis=1)
  df['NEW_LIVE_IND_STD'] = df[live].std(axis=1)
  df['NEW_LIVE_IND_KURT'] = df[live].kurtosis(axis=1)
  
  df['NEW_INC_PER_CHLD'] = df['AMT_INCOME_TOTAL'] / (1 + df['CNT_CHILDREN'])
  df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
  df['NEW_ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / (1 + df['AMT_INCOME_TOTAL'])
  df['NEW_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
  df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
  df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
  df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())
  df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
  df['NEW_CAR_TO_EMPLOY_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
  df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
  df['NEW_PHONE_TO_EMPLOY_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_EMPLOYED']
  df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

In [0]:
def process_log_df(df, col_prefix, group_col, sort_col, cat_cols, cont_cols):
  def second(ser):
    return ser.iloc[1] if len(ser) >= 2 else None

  def third(ser):
    return ser.iloc[2] if len(ser) >= 3 else None

  def second_last(ser):
    return ser.iloc[-1] if len(ser) >= 2 else None

  def third_last(ser):
    return ser.iloc[-2] if len(ser) >= 3 else None  
  
  
  FILE_PATH='./prep/processed_%s.snappy.parquet'%col_prefix
  
#   import os
#   if os.path.exists(FILE_PATH):
#     return pd.read_parquet(FILE_PATH)

#   cat_cols = cat_cols[:1]
#   cont_cols = cont_cols[:1]
  
  from tqdm import tqdm
  
  grouped = df.sort_values([group_col, sort_col]).groupby(group_col)
  df = grouped.size().to_frame()
  df.columns=['SIZE']
#   display(df.head())
  
  print("Processing categorical columns...")
  for col in tqdm(cat_cols):
    agg = grouped.agg({col: ['count', 'first', second, second_last, 'last']})
    agg.columns = [v[0] + '__' + v[1] + '__CATEGORY' if v[1] != 'count' else v[0] + '__' + v[1] for v in agg.columns.values]
    df = pd.concat([df, agg], axis=1)
    df = pd.concat([df, grouped[col].value_counts().unstack().fillna(0).astype(int).rename(lambda x: col + '__' + x, axis=1)], axis=1)    
#     display(df.head())
  
  print("Processing continous columns...")
  for col in tqdm(cont_cols):
    agg = grouped.agg({col: ['min', 'max', 'median', 'var', 'count', 'sum', 'first', second, second_last, 'last']})
    agg.columns = [v[0] + '__' + v[1] for v in agg.columns.values]
    df = pd.concat([df, agg], axis=1)
#     display(df.head())
  
  df.rename(lambda x: col_prefix + '__' + x, inplace=True, axis=1)  
#   display(df.head())
  df.to_parquet(FILE_PATH)
  return df

In [0]:
prev_app['APP_CREDIT_PERC'] = prev_app['AMT_APPLICATION'] / prev_app['AMT_CREDIT']

prev_app_cat_cols = [
    'NAME_CONTRACT_TYPE', 
    'WEEKDAY_APPR_PROCESS_START', 
    'FLAG_LAST_APPL_PER_CONTRACT', 
    'NAME_CASH_LOAN_PURPOSE', 
    'NAME_CONTRACT_STATUS', 
    'NAME_PAYMENT_TYPE', 
    'CODE_REJECT_REASON',
    'NAME_TYPE_SUITE',
    'NAME_CLIENT_TYPE',
    'NAME_GOODS_CATEGORY',
    'NAME_PORTFOLIO',
    'NAME_PRODUCT_TYPE',
    'CHANNEL_TYPE',
    'NAME_SELLER_INDUSTRY',
    'NAME_YIELD_GROUP',
    'PRODUCT_COMBINATION'
]
prev_app_cont_cols = prev_app.columns.drop(prev_app_cat_cols + ['SK_ID_CURR'])

processed_prev_app = process_log_df(
    prev_app, 
    col_prefix='PREV_APP', 
    group_col='SK_ID_CURR', 
    sort_col='DAYS_DECISION',
    cat_cols=prev_app_cat_cols,
    cont_cols=prev_app_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_prev_app, left_on='SK_ID_CURR', right_index=True, how='left')

  0%|          | 0/16 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/21 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 21/21 [06:12<00:00, 17.72s/it]


In [0]:
bureau_cat_cols = ['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
bureau_cont_cols = bureau.columns.drop(bureau_cat_cols + ['SK_ID_CURR'])

processed_bureau = process_log_df(
    bureau, 
    col_prefix='BUREAU', 
    group_col='SK_ID_CURR',
    sort_col='DAYS_CREDIT',
    cat_cols=bureau_cat_cols,
    cont_cols=bureau_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_bureau, left_on='SK_ID_CURR', right_index=True, how='left')

  0%|          | 0/3 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/13 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 13/13 [03:02<00:00, 14.06s/it]


In [0]:
pd.set_option('display.max_columns', None)
dfs['train'].head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,PREV_APP__SIZE,PREV_APP__NAME_CONTRACT_TYPE__count,PREV_APP__NAME_CONTRACT_TYPE__first__CATEGORY,PREV_APP__NAME_CONTRACT_TYPE__second__CATEGORY,PREV_APP__NAME_CONTRACT_TYPE__second_last__CATEGORY,PREV_APP__NAME_CONTRACT_TYPE__last__CATEGORY,PREV_APP__NAME_CONTRACT_TYPE__Cash loans,PREV_APP__NAME_CONTRACT_TYPE__Consumer loans,PREV_APP__NAME_CONTRACT_TYPE__Revolving loans,PREV_APP__NAME_CONTRACT_TYPE__XNA,PREV_APP__WEEKDAY_APPR_PROCESS_START__count,PREV_APP__WEEKDAY_APPR_PROCESS_START__first__CATEGORY,PREV_APP__WEEKDAY_APPR_PROCESS_START__second__CATEGORY,PREV_APP__WEEKDAY_APPR_PROCESS_START__second_last__CATEGORY,PREV_APP__WEEKDAY_APPR_PROCESS_START__last__CATEGORY,PREV_APP__WEEKDAY_APPR_PROCESS_START__FRIDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__MONDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__SATURDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__SUNDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__THURSDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__TUESDAY,PREV_APP__WEEKDAY_APPR_PROCESS_START__WEDNESDAY,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__count,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__first__CATEGORY,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__second__CATEGORY,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__second_last__CATEGORY,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__last__CATEGORY,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__N,PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__Y,PREV_APP__NAME_CASH_LOAN_PURPOSE__count,PREV_APP__NAME_CASH_LOAN_PURPOSE__first__CATEGORY,PREV_APP__NAME_CASH_LOAN_PURPOSE__second__CATEGORY,PREV_APP__NAME_CASH_LOAN_PURPOSE__second_last__CATEGORY,PREV_APP__NAME_CASH_LOAN_PURPOSE__last__CATEGORY,PREV_APP__NAME_CASH_LOAN_PURPOSE__Building a house or an annex,PREV_APP__NAME_CASH_LOAN_PURPOSE__Business development,PREV_APP__NAME_CASH_LOAN_PURPOSE__Buying a garage,PREV_APP__NAME_CASH_LOAN_PURPOSE__B

In [0]:
bureau_balance_cat_cols = ['STATUS']
bureau_balance_cont_cols = bureau_balance.columns.drop(bureau_balance_cat_cols + ['SK_ID_BUREAU'])
processed_bureau_balance = process_log_df(
    bureau_balance, 
    'BUREAU_BALANCE', 
    'SK_ID_BUREAU', 
    'MONTHS_BALANCE', 
    cat_cols=bureau_balance_cat_cols, 
    cont_cols=bureau_balance_cont_cols
)
for t in types:
  dfs[t] = pd.merge(dfs[t], processed_bureau_balance, left_on='BUREAU__SK_ID_BUREAU__last', right_index=True, how='left')

  0%|          | 0/1 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/1 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 1/1 [00:42<00:00, 42.00s/it]


In [0]:
pos_cash_cat_cols = ['NAME_CONTRACT_STATUS']
pos_cash_cont_cols = pos_cash.columns.drop(pos_cash_cat_cols + ['SK_ID_CURR'])
processed_pos_cash = process_log_df(
    pos_cash, 
    'POS_CASH', 
    'SK_ID_CURR', 
    'MONTHS_BALANCE', 
    cat_cols=pos_cash_cat_cols, 
    cont_cols=pos_cash_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_pos_cash, left_on='SK_ID_CURR', right_index=True, how='left')

  0%|          | 0/1 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/6 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 6/6 [01:39<00:00, 16.63s/it]


In [0]:
credit_card_balance_cat_cols = ['NAME_CONTRACT_STATUS']
credit_card_balance_cont_cols = credit_card_balance.columns.drop(credit_card_balance_cat_cols + ['SK_ID_CURR'])
processed_credit_card_balance = process_log_df(
    credit_card_balance, 
    'CREDIT_CARD_BALANCE', 
    'SK_ID_CURR', 
    'MONTHS_BALANCE', 
    cat_cols=credit_card_balance_cat_cols, 
    cont_cols=credit_card_balance_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_credit_card_balance, left_on='SK_ID_CURR', right_index=True, how='left')

  0%|          | 0/1 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/21 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 21/21 [02:08<00:00,  6.13s/it]


In [0]:
previous_application_cat_cols = [
    'NAME_CONTRACT_TYPE', 
    'WEEKDAY_APPR_PROCESS_START', 
    'FLAG_LAST_APPL_PER_CONTRACT', 
    'NAME_CONTRACT_STATUS', 
    'NAME_PAYMENT_TYPE', 
    'CODE_REJECT_REASON', 
    'NAME_TYPE_SUITE', 
    'NAME_CLIENT_TYPE', 
    'NAME_GOODS_CATEGORY', 
    'NAME_PORTFOLIO', 
    'NAME_PRODUCT_TYPE', 
    'CHANNEL_TYPE', 
    'NAME_SELLER_INDUSTRY', 
    'NAME_YIELD_GROUP', 
    'PRODUCT_COMBINATION',
    'NAME_CASH_LOAN_PURPOSE']
previous_application_cont_cols = previous_application.columns.drop(previous_application_cat_cols + ['SK_ID_CURR'])
processed_previous_application = process_log_df(
    previous_application, 
    'PREVIOUS_APPLICATION', 
    'SK_ID_CURR', 
    'DAYS_DECISION', 
    cat_cols=previous_application_cat_cols, 
    cont_cols=previous_application_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_previous_application, left_on='SK_ID_CURR', right_index=True, how='left')

  0%|          | 0/16 [00:00<?, ?it/s]

Processing categorical columns...


  0%|          | 0/20 [00:00<?, ?it/s]

Processing continous columns...


100%|██████████| 20/20 [06:22<00:00, 19.14s/it]


In [0]:
installments_payments['PAYMENT_PERC'] = installments_payments['AMT_PAYMENT'] / installments_payments['AMT_INSTALMENT']
installments_payments['PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']
installments_payments['DPD'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
installments_payments['DBD'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['DPD'] = installments_payments['DPD'].apply(lambda x: x if x > 0 else 0)
installments_payments['DBD'] = installments_payments['DBD'].apply(lambda x: x if x > 0 else 0)

installments_payments_cat_cols = []
installments_payments_cont_cols = installments_payments.columns.drop(installments_payments_cat_cols + ['SK_ID_CURR'])
processed_installments_payments = process_log_df(
    installments_payments, 
    'INSTALLMENTS_PAYMENTS', 
    'SK_ID_CURR', 
    'DAYS_INSTALMENT', 
    cat_cols=installments_payments_cat_cols, 
    cont_cols=installments_payments_cont_cols
)

for t in types:
  dfs[t] = pd.merge(dfs[t], processed_installments_payments, left_on='SK_ID_CURR', right_index=True, how='left')

0it [00:00, ?it/s]
  0%|          | 0/11 [00:00<?, ?it/s]

Processing categorical columns...
Processing continous columns...


100%|██████████| 11/11 [03:44<00:00, 20.45s/it]


In [0]:
ext_source_cols = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
impute_ext_source_cols = ['IMPUTE_EXT_SOURCE_1', 'IMPUTE_EXT_SOURCE_2', 'IMPUTE_EXT_SOURCE_3']

for t in types:
  dfs[t]['EXT_SOURCE_count'] = dfs[t][ext_source_cols].count(axis=1)
  dfs[t]['EXT_SOURCE_max'] = dfs[t][ext_source_cols].max(axis=1)
  dfs[t]['EXT_SOURCE_min'] = dfs[t][ext_source_cols].min(axis=1)
  dfs[t]['EXT_SOURCE_median'] = dfs[t][ext_source_cols].median(axis=1)
  dfs[t]['EXT_SOURCE_diff'] = dfs[t]['EXT_SOURCE_max'] - dfs[t]['EXT_SOURCE_min']
  dfs[t][impute_ext_source_cols] = dfs[t][ext_source_cols]

/usr/local/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [0]:
# imputer for handling missing values
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

# Need to impute missing values
dfs['train'][impute_ext_source_cols] = imputer.fit_transform(dfs['train'][impute_ext_source_cols])
dfs['test'][impute_ext_source_cols] = imputer.transform(dfs['test'][impute_ext_source_cols])

from sklearn.preprocessing import PolynomialFeatures
                                  
# Create the polynomial object with specified degree
poly_transformer = PolynomialFeatures(degree = 3)
poly_transformer.fit(dfs['train'][impute_ext_source_cols])
feats_name = ['IMPUTE_POLY_FEATS_'+col for col in poly_transformer.get_feature_names(input_features = impute_ext_source_cols)]

for t in types:
  dfs[t][feats_name] = pd.DataFrame(poly_transformer.transform(dfs[t][impute_ext_source_cols]), columns = feats_name)

In [0]:
bool_keywords = ['FLAG', 'NOT']

for keyword in bool_keywords:
  doc_cols = []
  for col in dfs['train']:
    if keyword in col:
      doc_cols.append(col)

  print(doc_cols)
  for t in types:
    dfs[t][keyword+'_total'] = dfs[t][doc_cols].sum(axis=1)
#     dfs[t].drop(columns=doc_cols, inplace=True)

['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__count', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__first__CATEGORY', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__second__CATEGORY', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__second_last__CATEGORY', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__last__CATEGORY', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__N', 'PREV_APP__FLAG_LAST_APPL_PER_CONTRACT__Y', 'PREV_APP__NFLAG_LAST_APPL_IN_DAY__min', 'PREV_APP__NFLAG_LAST_APPL_IN_DAY__max', 'PREV_APP__NFLAG_

In [0]:
for t in types:
  dfs[t].to_parquet('./prep/application_%s.snappy.parquet'%t)
  
!gsutil rsync ./prep gs://{os.environ['GCP_BUCKET']}/prep

Building synchronization state...
Starting synchronization...
Copying file://./prep/application_test.snappy.parquet [Content-Type=application/octet-stream]...
Copying file://./prep/application_train.snappy.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://./prep/processed_BUREAU.snappy.parquet [Content-Type